<a href="https://colab.research.google.com/github/iraidaantropova/ApacheSpark/blob/main/HW4_PySpark_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyspark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

Задание 1
Добавьте к таблице следующие поля:

Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
Стоимость последнего проданного дома до текущего

In [6]:
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType

import pandas as pd
import requests

In [7]:
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)
df.printSchema()
df.show(truncate=False)

root
 |-- datesold: timestamp (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)

+-------------------+--------+-------+------------+--------+
|datesold           |postcode|price  |propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|2607    |525000 |house       |4       |
|2007-02-27 00:00:00|2906    |290000 |house       |3       |
|2007-03-07 00:00:00|2905    |328000 |house       |3       |
|2007-03-09 00:00:00|2905    |380000 |house       |4       |
|2007-03-21 00:00:00|2906    |310000 |house       |3       |
|2007-04-04 00:00:00|2905    |465000 |house       |4       |
|2007-04-24 00:00:00|2607    |399000 |house       |3       |
|2007-04-30 00:00:00|2606    |1530000|house       |4       |
|2007-05-24 00:00:00|2902    |359000 |house       |3       |
|2007-05-25 00:00:00|2906    |320000 |house       |3   

In [11]:
from pyspark.sql.functions import col, asc,desc

#провожу группировку данных по postcode, сортирую по дате продажи
windSpec_before = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')

#получаю среднюю стоимость домов до текущей продажи
# среднее найдено с помощью avg()
df = df.withColumn('average_price_before', F.avg('price').over(windSpec_before))

In [12]:
windSpec_after = Window()\
    .partitionBy('postcode')\
    .orderBy(col('datesold').desc())

df = df.withColumn('average_price_after', F.avg('price').over(windSpec_after))

In [13]:
df = df.withColumn('previous_sold_price', F.lag('price', 1).over(windSpec_before))

In [14]:
df.show(10)

+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+
|           datesold|postcode|  price|propertyType|bedrooms|last_price_before_sale|average_price_before|average_price_after|previous_sold_price|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            327000.0| 1203842.0951327435|               null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                327000|            558500.0|  1205786.312638581|             327000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                790000|   647333.3333333334| 1206710.2822222223|             790000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|                315000|            558625.0|  1207560.416481069|      

В итоге у вас таблица с колонками (или нечто похожее):

price
Среднегодовая цена
Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
Стоимость последнего проданного дома до текущего ((1 балл)
и др.
Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [20]:
@F.udf(returnType=IntegerType())
def count_unique_udf(row):
    return len(set(row))
(
    df.filter(F.col('average_price_before').isNotNull())
    .withColumn('unique_values', count_unique_udf(F.array(*df))).show(10)

)

+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+-------------+
|           datesold|postcode|  price|propertyType|bedrooms|last_price_before_sale|average_price_before|average_price_after|previous_sold_price|unique_values|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+-------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            327000.0| 1203842.0951327435|               null|            8|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                327000|            558500.0|  1205786.312638581|             327000|            8|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                790000|   647333.3333333334| 1206710.2822222223|             790000|            8|
|2008-04-24 00:00:00|    2600| 292500|       h

In [21]:
@pandas_udf(IntegerType(), PandasUDFType.SCALAR)
def count_unique_pandas_udf(series) -> int:
    return series.apply(lambda row: len(set(row)))
(
    df.filter(F.col('average_price_before').isNotNull())
    .withColumn('unique_values', count_unique_pandas_udf(F.array(*df))).show(10)
)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+-------------+
|           datesold|postcode|  price|propertyType|bedrooms|last_price_before_sale|average_price_before|average_price_after|previous_sold_price|unique_values|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+-------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            327000.0| 1203842.0951327435|               null|            8|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                327000|            558500.0|  1205786.312638581|             327000|            8|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                790000|   647333.3333333334| 1206710.2822222223|             790000|            8|
|2008-04-24 00:00:00|    2600| 292500|       h

Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [22]:
# использую функцию when(если TRUE, тогда)
df = df.withColumn(
    'sign',
    F.when(
        F.col('average_price_before').isNull() |
        F.col('average_price_after').isNull(),
        'Нет данных'
    ).when(
        F.col('average_price_before') > F.col('average_price_after'),
        '-'
    ).when(
        F.col('average_price_before') < F.col('average_price_after'),
        '+'
    ).otherwise(
        '='
    )
)

df.show()

+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+----+
|           datesold|postcode|  price|propertyType|bedrooms|last_price_before_sale|average_price_before|average_price_after|previous_sold_price|sign|
+-------------------+--------+-------+------------+--------+----------------------+--------------------+-------------------+-------------------+----+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|            327000.0| 1203842.0951327435|               null|   +|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|                327000|            558500.0|  1205786.312638581|             327000|   +|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|                790000|   647333.3333333334| 1206710.2822222223|             790000|   +|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|                315000|            55862